# Answering Business Questions using SQL

In this guided project I'm going to use my SQL skills to answer business questions.

I'll be working with a version of a database called [Chinook](https://github.com/lerocha/chinook-database). The Chinook database contains information about a fictional digital music shop - kind of like a mini-iTunes store.

The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables. The schema diagram help understand the available columns and the structure of the data. Here's a schema diagram for the Chinook database:

![Chinook schema diagram](https://s3.amazonaws.com/dq-content/190/chinook-schema.svg)

# Introduction

The Chinook database is provided as a SQLite database file called `chinook.db`

In [0]:
# Import libraries
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Creating Helper Functions

In [0]:
#run_query(q): Takes a SQL query as an argument and returns a pandas dataframe by using the connection as a SQLite built-in context manager. 
def run_query(q):
    with sqlite3.connect('/content/chinook.db') as conn:
        return pd.read_sql_query(q, conn)
    
#run_command(c): Takes a SQL command as an argument and executes it using the sqlite module.
def run_command(c):
    with sqlite3.connect('/content/chinook.db') as conn:
        conn.isolation_level = None
        conn.execute(c)
    
#show_tables(): calls the run_query() function to return a list of all tables and views in the database.
def show_tables():
    q = '''SELECT
            name,
            type
        FROM sqlite_master
        WHERE type IN ("table","view");
        '''
    return run_query(q) 

Let's start by getting familiar with the data showing a list of all tables and views in the database.

In [88]:
#Initial state of the database
show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


# Analysis

## Top Selling Genres in USA

Let's assume that the Chinook record store has just signed a deal with a new record label, and I've been tasked with selecting the first three albums that will be added to the store. All albums are by artists that don't have any tracks in the store right now, but we have the genre of music they produce:

Artist Name | Genre
--- | ---
Regal |Hip-Pop
Red Tone | Punk
Meteor and the Girls | Pop
Slim Jim Bites |	Blues

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [89]:
q1 = '''
WITH invoice_usa AS (

  SELECT
    invoice_line.track_id,
    invoice_line.quantity
  FROM invoice_line
  INNER JOIN invoice
    ON invoice_line.invoice_id = invoice.invoice_id
  INNER JOIN customer
    ON invoice.customer_id = customer.customer_id
  WHERE customer.country = "USA"

), genre_quantity AS (

  SELECT
    genre.name,
    SUM(invoice_usa.quantity)                                               AS sum_quantity,
    SUM(invoice_usa.quantity) * 100.0 / (SELECT COUNT(*) FROM invoice_usa)  AS perc_quantity
  FROM genre
  INNER JOIN track
    ON genre.genre_id = track.genre_id
  INNER JOIN invoice_usa
    ON track.track_id = invoice_usa.track_id
  GROUP BY 1
  ORDER BY 2 DESC
  LIMIT 10
  
)


SELECT *
FROM genre_quantity;
'''

run_query(q1)

,name,sum_quantity,perc_quantity
0,Rock,561,53.377735
1,Alternative & Punk,130,12.369172
2,Metal,124,11.798287
3,R&B/Soul,53,5.042816
4,Blues,36,3.425309
5,Alternative,35,3.330162
6,Latin,22,2.093245
7,Pop,22,2.093245
8,Hip Hop/Rap,20,1.902950
9,Jazz,14,1.332065


Based on the sales of tracks across different genres in the USA, the company should purchase new albums by the following artists:

- Red Tone (Punk)
- Slim Jim Bites (Blues)
- Meteor and the Girls (Pop)

Besides the artists currently signed to that label, the company should stay tuned and pay attention to upcoming artists in the `Rock` genre, since compared to the 18% the previous four genres add up to, it represents a 53% of the store total sales.

## Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. Let's assume that I have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [90]:
q2 = '''
WITH sales_dollars AS (

    SELECT
        employee.first_name || " " || employee.last_name  AS employee_name,
        employee.hire_date,
        invoice.total
    FROM employee
    INNER JOIN customer
      ON employee.employee_id = customer.support_rep_id
    INNER JOIN invoice
      ON customer.customer_id = invoice.customer_id
    WHERE employee.title = "Sales Support Agent"   

), final AS (

    SELECT 
        employee_name,
        hire_date,
        SUM(total)                                                   AS total,
        SUM(total) * 100.0 / (SELECT SUM(total) FROM sales_dollars)  AS total_perc
    FROM sales_dollars
    GROUP BY 1, 2
    ORDER BY total DESC

)


SELECT *
FROM final;
'''

run_query(q2)

,employee_name,hire_date,total,total_perc
0,Jane Peacock,2017-04-01 00:00:00,1731.51,36.766870
1,Margaret Park,2017-05-03 00:00:00,1584.00,33.634644
2,Steve Johnson,2017-10-17 00:00:00,1393.92,29.598486


Jane Peacock has made the highest amount of sales so far, with a 36.8% of the total, followed by Margaret Park and Steve Johnson, with a 33.6% and 29.6% respectively.

Even though there is just a slight difference in the percentage of the total sales amount between all three agents, in terms of total amount in dollars, the best agent, Jane Peacock, has sold about 20% more than the last agent, Steve Johnson.

Taking a look at the previous table, we can conclude that the difference in sales correlates with the time each employee has been working in the company, i.e. their hiring dates.

## Analyzing Sales by Country

The next goal is to analyze the sales data for customers from each different country.

In [91]:
q3 = '''
WITH sales_customers AS (

    SELECT
        customer.customer_id,
        customer.country,
        invoice.total
    FROM customer
    INNER JOIN invoice
      ON customer.customer_id = invoice.customer_id

), tot_customers AS (

    SELECT
        customer_id,
        country,
        SUM(total) AS total
    FROM sales_customers
    GROUP BY 1

), num_customers AS (

    SELECT
        country,
        COUNT(customer_id) AS num_customer
    FROM tot_customers
    GROUP BY 1

), other AS (

    SELECT
        country
    FROM num_customers
    WHERE num_customer = 1

), sub_sales AS (

    SELECT
        customer_id,
        CASE
            WHEN country IN other THEN "Other"
            ELSE country
        END AS country,
        CASE
            WHEN country IN other THEN 1
            ELSE 0
        END AS sort,
        total
    FROM sales_customers

), order_by_country AS (

    SELECT
        country,
        COUNT(DISTINCT customer_id)   AS num_customer,
        AVG(total)                    AS order_avg,
        SUM(total)                    AS total_sales,
        sort
    FROM sub_sales
    GROUP BY 1

), order_by_customer AS (

    SELECT
        customer_id,
        country,
        SUM(total) AS total
    FROM sub_sales
    GROUP BY 1

), customer_value AS (

    SELECT
        country,
        AVG(total) AS customer_value_avg
    FROM order_by_customer
    GROUP BY 1

), final AS(

    SELECT
        order_by_country.country,
        order_by_country.num_customer,
        order_by_country.order_avg,
        order_by_country.total_sales,
        customer_value.customer_value_avg
    FROM order_by_country
    INNER JOIN customer_value
        ON order_by_country.country = customer_value.country
    ORDER BY sort ASC, 4 DESC

)



SELECT *
FROM final;
'''

run_query(q3)

,country,num_customer,order_avg,total_sales,customer_value_avg
0,USA,13,7.942672,1040.49,80.037692
1,Canada,8,7.047237,535.59,66.948750
2,Brazil,5,7.011148,427.68,85.536000
3,France,5,7.781400,389.07,77.814000
4,Germany,4,8.161463,334.62,83.655000
5,Czech Republic,2,9.108000,273.24,136.620000
6,United Kingdom,3,8.768571,245.52,81.840000
7,Portugal,2,6.383793,185.13,92.565000
8,India,2,8.721429,183.15,91.575000
9,Other,15,7.448571,1094.94,72.996000


Looking at the results from the data, based on the total amount of sales, USA and Canada are the biggest markets for the company, followed by countries like Brazil or France.

However, based on the number of customers, what unique customers have spent so far, and the average order values, Czech Republic, the UK, India and Portugal show a big potential for growth, since they have less customers that, on average, spend a higher total and per order amount of money.

It should be pointed out that based on the sample size, in order for these data references to be more accurate and give the advertising team more confidence, I would advise the company to start out by running smaller marketing campaigns in those countries, collecting data from new customers and scaling up once the results are proved to be consistent with the current trends.

## Albums vs Individual Tracks

Management of the Chinook store are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

Let's assume that I have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In this instance, there are two edge cases to consider:

- Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
- Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since the analysis is concerned with maximizing revenue I can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.

In [92]:
q4 = '''
WITH base_table AS (

    SELECT 
        invoice_line.invoice_id,
        invoice_line.track_id,
        track.album_id
    FROM invoice_line
    INNER JOIN track
        ON invoice_line.track_id = track.track_id

), album_info AS (

    SELECT
        album_id,
        COUNT(DISTINCT track_id) AS track_album
    FROM base_table
    GROUP BY 1
    -- HAVING COUNT(DISTINCT track_id) > 1
    ORDER BY 2 DESC

), invoice_info AS (

    SELECT
        invoice_id,
        album_id,
        COUNT(DISTINCT track_id) AS track_buyed
    FROM base_table
    GROUP BY 1, 2
    ORDER BY 3 DESC

), more_invoice_info AS (

    SELECT
        invoice_info.*,
        album_info.track_album
    FROM invoice_info
    LEFT JOIN album_info
        ON invoice_info.album_id = album_info.album_id
    
), album_buyed AS (

    SELECT
        more_invoice_info.invoice_id,
        CASE
            WHEN more_invoice_info.track_buyed = more_invoice_info.track_album THEN "yes"
            ELSE "no"
        END AS album_purchase
    FROM more_invoice_info

), album_buyed_remove_single AS (

    SELECT
        album_buyed.invoice_id,
        MIN(album_purchase) AS album_purchase
    FROM album_buyed
    GROUP BY 1

), final AS (

    SELECT
        album_purchase,
        COUNT(DISTINCT invoice_id)                                                                               AS number_of_invoices,
        COUNT(DISTINCT invoice_id) * 100.0 / (SELECT COUNT(DISTINCT invoice_id) FROM album_buyed_remove_single)  AS album_purchase_perc
    FROM album_buyed_remove_single
    GROUP BY 1     

)


SELECT *
FROM final;
'''

run_query(q4)

,album_purchase,number_of_invoices,album_purchase_perc
0,no,500,81.433225
1,yes,114,18.566775


I would suggest keeping the current strategy, since allowing users to buy only the most popular tracks from albums from record companies could potentially reduce the number of individual track purchases, which right now represents one fifth of the total revenue.